In [ ]:
from pdfminer.high_level import extract_text
import re
import unicodedata



In [ ]:
def p(texto: str) -> str:
    """ Padroniza texto: deixa tudo minusculo, remove pontuacoes, remove acentuacao"""

    texto = texto.lower().strip(",.;")
    return ''.join(
        c for c in unicodedata.normalize('NFD', texto)
        if unicodedata.category(c) != 'Mn'
    )


In [ ]:

# Relatorios Simplificado
caminho = "files/PR-KNM_16_08_2023_PUB.pdf"
# caminho = "files/PT-UCK_03_02_2024-PUB..pdf"

# Relatorios Completo (Em desenvolvimento)
# caminho = "files/PR_TTO_24_07_2024_PUB.1.pdf"

texto = extract_text(caminho)

len(texto)

In [ ]:
# print(texto)

In [ ]:
padrao = r"3\.2 Fatores Contribuintes(.*?)4\. RECOMENDAÇÕES DE SEGURANÇA"
labels_raw = re.search(padrao, texto, re.S)

if labels_raw:
    print(labels_raw.group(1))
    print("-----------")

    linhas = [l.strip(" -;\n") for l in labels_raw.group(1).splitlines() if l.strip()]
    dicionario = {}
    for l in linhas:
        partes = re.split(r"\s[–-]\s", l, maxsplit=1)  # aceita – ou -
        if len(partes) == 2:
            chave, valor_raw = partes
            valor = p(valor_raw[:1])  # Decidir se 1, 2 ou 2, 1
            dicionario[p(chave)] = valor

    print(dicionario)
else:
    print("Trecho não encontrado")